
# Swing Strategy Scanner — Williams %R + CCI + EMA (NIFTY500-ready)

This notebook scans a list of stocks (e.g., the NIFTY500 universe) and filters those that currently satisfy the swing-trading strategy based on:
- **Trend**: 50-day EMA
- **Momentum Extremes**: Williams %R (14)
- **Deviation Confirmation**: CCI (20)

It produces two outputs:
- `long_candidates.csv` — stocks meeting **long** setup conditions
- `short_candidates.csv` — stocks meeting **short** setup conditions

> The rules are implemented exactly as described in your uploaded strategy note.  fileciteturn0file0


## 1) Setup — install & imports

In [13]:

# If needed, uncomment and run:
# %pip install pandas numpy yfinance

import math, sys, os, warnings, json, time, datetime as dt
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict

import numpy as np
import pandas as pd
import yfinance as yf  # data


## 2) Parameters — universe & indicator settings

In [14]:

# === Universe ===
# Put your stock list here; for NSE, use '.NS' (e.g., RELIANCE.NS)
TICKERS = ['ZENSARTECH.NS']

HISTORY_START = "2018-01-01"
MIN_BARS = 250  # require at least this many bars after indicators

# === Indicators ===
EMA_LEN = 50      # trend filter
WR_LEN  = 14      # Williams %R
CCI_LEN = 20      # CCI
RECENT_LOOKBACK = 15  # "recently crossed" window for confirmations

# === Output ===
OUT_LONG  = "long_candidates.csv"
OUT_SHORT = "short_candidates.csv"
OUT_NONE  = "no_signal_or_errors.csv"


## 3) Indicator helpers

In [15]:

def ema(series: pd.Series, length: int) -> pd.Series:
    return series.ewm(span=length, adjust=False, min_periods=length).mean()

def williams_r(h: pd.Series, l: pd.Series, c: pd.Series, length: int) -> pd.Series:
    hh = h.rolling(length).max()
    ll = l.rolling(length).min()
    return -100 * (hh - c) / (hh - ll)  # 0 to -100

def cci(h: pd.Series, l: pd.Series, c: pd.Series, length: int) -> pd.Series:
    typical = (h + l + c) / 3.0
    sma = typical.rolling(length).mean()
    mad = (typical - sma).abs().rolling(length).mean()
    return (typical - sma) / (0.015 * mad)


## 4) Strategy logic — rules & helpers

In [16]:

@dataclass
class SignalResult:
    side: str              # "LONG" or "SHORT"
    date: pd.Timestamp
    close: float
    ema50: float
    wr14: float
    cci20: float
    notes: str

def recent_cross_above(series: pd.Series, level: float, lookback: int) -> bool:
    """True if series moved from <= level to > level within 'lookback' bars and is > level now."""
    if len(series) < lookback + 2:
        return False
    wnd = series.iloc[-(lookback+1):]
    crossed_before = (wnd.shift(1) <= level).any()
    now_above = wnd.iloc[-1] > level
    return bool(crossed_before and now_above)

def recent_cross_below(series: pd.Series, level: float, lookback: int) -> bool:
    """True if series moved from >= level to < level within 'lookback' bars and is < level now."""
    if len(series) < lookback + 2:
        return False
    wnd = series.iloc[-(lookback+1):]
    crossed_before = (wnd.shift(1) >= level).any()
    now_below = wnd.iloc[-1] < level
    return bool(crossed_before and now_below)

def compute_indicators(hist: pd.DataFrame) -> pd.DataFrame:
    df = hist.copy()
    if df.empty:
        return df
    df["EMA50"] = ema(df["Close"], EMA_LEN)
    df["WR14"]  = williams_r(df["High"], df["Low"], df["Close"], WR_LEN)
    df["CCI20"] = cci(df["High"], df["Low"], df["Close"], CCI_LEN)
    return df.dropna().copy()

def scan_last_bar(df: pd.DataFrame) -> Tuple[bool, str, Optional[SignalResult]]:
    """Apply entry rules to the last bar only."""
    if df.empty: 
        return False, "empty df", None
    last = df.iloc[-1]
    close, ema50, wr, cci_val = float(last["Close"]), float(last["EMA50"]), float(last["WR14"]), float(last["CCI20"])
    wr_series = df["WR14"]
    cci_series = df["CCI20"]

    # LONG: Uptrend + momentum turn after oversold
    long_cond_trend = close > ema50
    long_cond_wr = (wr_series.iloc[-RECENT_LOOKBACK:].min() < -80) and recent_cross_above(wr_series, -50, RECENT_LOOKBACK)
    long_cond_cci = (cci_series.iloc[-RECENT_LOOKBACK:].min() < -100) and (cci_val >= -100)

    if long_cond_trend and long_cond_wr and long_cond_cci:
        return True, "LONG", SignalResult(
            "LONG", df.index[-1], close, ema50, wr, cci_val,
            "Uptrend; %R<-80 then >-50; CCI<-100 then >=-100"
        )

    # SHORT: Downtrend + momentum turn after overbought
    short_cond_trend = close < ema50
    short_cond_wr = (wr_series.iloc[-RECENT_LOOKBACK:].max() > -20) and recent_cross_below(wr_series, -50, RECENT_LOOKBACK)
    short_cond_cci = (cci_series.iloc[-RECENT_LOOKBACK:].max() > +100) and (cci_val <= +100)

    if short_cond_trend and short_cond_wr and short_cond_cci:
        return True, "SHORT", SignalResult(
            "SHORT", df.index[-1], close, ema50, wr, cci_val,
            "Downtrend; %R>-20 then <-50; CCI>+100 then <=+100"
        )

    return False, "NONE", None


## 5) Data download — yfinance helper

In [17]:

def get_hist(ticker: str, start: str) -> pd.DataFrame:
    """Download daily OHLCV for ticker since 'start'."""
    try:
        df = yf.download(ticker, start=start, progress=False, auto_adjust=False, multi_level_index=False)
        if df is None or df.empty:
            return pd.DataFrame()
        for col in ["Open","High","Low","Close","Adj Close","Volume"]:
            if col not in df.columns:
                return pd.DataFrame()
        return df
    except Exception as e:
        print(f"[ERR] {ticker}: {e}")
        return pd.DataFrame()


## 6) Run scanner — filter current signals & write CSVs

In [18]:

rows_long, rows_short, rows_none = [], [], []

for t in TICKERS:
    hist = get_hist(t, HISTORY_START)
    if hist.empty or len(hist) < MIN_BARS:
        rows_none.append({"Ticker": t, "Status": "insufficient data"})
        continue

    df = compute_indicators(hist)
    if df.empty or len(df) < MIN_BARS:
        rows_none.append({"Ticker": t, "Status": "insufficient bars post-indicator"})
        continue

    ok, side, sig = scan_last_bar(df)
    if not ok:
        rows_none.append({"Ticker": t, "Status": side})
        continue

    row = {
        "Ticker": t,
        "Date": sig.date.date(),
        "Close": round(sig.close, 4),
        "EMA50": round(sig.ema50, 4),
        "WR14": round(sig.wr14, 2),
        "CCI20": round(sig.cci20, 1),
        "Notes": sig.notes
    }
    if sig.side == "LONG":
        rows_long.append(row)
    elif sig.side == "SHORT":
        rows_short.append(row)

# Save CSVs
if rows_long:
    pd.DataFrame(rows_long).sort_values(["Date","Ticker"]).to_csv(OUT_LONG, index=False)
if rows_short:
    pd.DataFrame(rows_short).sort_values(["Date","Ticker"]).to_csv(OUT_SHORT, index=False)
if rows_none:
    pd.DataFrame(rows_none).to_csv(OUT_NONE, index=False)

# Display summaries (plain pandas)
def _show(df, title):
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))
    display(df)

if rows_long:
    _show(pd.DataFrame(rows_long).sort_values(["Date","Ticker"]), "Long candidates (current bar)")
else:
    print("No LONG candidates on the latest bar.")

if rows_short:
    _show(pd.DataFrame(rows_short).sort_values(["Date","Ticker"]), "Short candidates (current bar)")
else:
    print("No SHORT candidates on the latest bar.")

if rows_none:
    _show(pd.DataFrame(rows_none), "No-signal or errors")



Long candidates (current bar)


,Ticker,Date,Close,EMA50,WR14,CCI20,Notes
0,ZENSARTECH.NS,2025-09-19,846.45,808.0233,-21.24,148.5,Uptrend; %R<-80 then >-50; CCI<-100 then >=-100


No SHORT candidates on the latest bar.



## 7) Notes & Adjustments

- **Lookback logic**: We check that %R/CCI reached an extreme *recently* and has now flipped back through a mid-zone threshold (−50 for %R, ±100 for CCI) in the trend’s direction.
- **Recency window**: `RECENT_LOOKBACK = 15` bars by default. Increase it to be more forgiving; decrease to be stricter.
- **Trend filter**: 50-EMA is used as the intermediate trend for a 3–4 week horizon.
- **Liquidity**: You said _no constraints_ on price/volume. You can optionally add a liquidity filter (e.g., `Volume` median > some threshold) before scanning.
- **Execution**: Signals are evaluated on the **latest bar** only to make a clean actionable list.
- **Backtest**: If you’d like, I can extend this notebook with a simple vectorized backtest with ~20 trading-day hold and protective stop.
